In [ ]:
# 1. Proje Kurulumu ve Güncelleme (Colab stok + eksikler)
import os
import sys
import subprocess
import time

try:
    from google.colab import drive
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    print("Google Colab ortamı algılandı.")
    repo_url = "https://github.com/onndd/newproje.git"
    project_dir = "/content/newproje"

    if os.path.exists(project_dir):
        %cd {project_dir}
        !git pull
    else:
        !git clone {repo_url}
        %cd {project_dir}

    print("Gerekli kütüphaneler yükleniyor...")

    # AGGRESSIVE FIX: Clean old numpy/pandas first
    print("Cleaning conflicting libraries...")
    try:
        subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "numpy", "pandas"], check=False)
    except Exception as e:
        print(f"Uninstall notice (safe to ignore): {e}")

    print("Installing compatible numpy and pandas...")
    # Force reinstall with specific versions
    subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy<2.0", "pandas==2.2.2", "--force-reinstall", "--no-deps"])
    
    # Re-install dependencies needed that might have been removed or need check
    packages = [
        "catboost==1.2.8",
        "lightgbm==4.6.0",
        "hmmlearn==0.3.2",
        "optuna==3.5.0",
        "streamlit==1.29.0",
        "stable-baselines3==2.2.1",
        "shimmy==1.3.0",
        "gymnasium==0.29.1"
    ]
    
    for pkg in packages:
        print(f"Installing {pkg}...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
            print(f"Successfully installed {pkg}")
        except subprocess.CalledProcessError as e:
            print(f"❌ ERROR installing {pkg}: {e}")

    if project_dir not in sys.path:
        sys.path.append(project_dir)
    
    print("\n✅ KURULUM TAMAMLANDI.")
    print("🛑🛑 ÇOK ÖNEMLİ: ŞİMDİ 'RUNTIME > RESTART SESSION' YAPMALISINIZ 🛑🛑")
    print("     (Aksi takdirde 'numpy.dtype size changed' hatası gitmeyecektir.)")
else:
    print("Local environment.")
    project_dir = os.getcwd()
    if project_dir not in sys.path:
        sys.path.append(project_dir)


In [ ]:
# 2. Veri Yükleme ve 3'lü Ayrım (3-Way Split)
import sys
import os
if '/content/newproje' not in sys.path:
    sys.path.append('/content/newproje')
import sqlite3
import pandas as pd
import numpy as np
from jetx_project.data_loader import load_data
from jetx_project.config import DB_PATH, PROFIT_SCORING_WEIGHTS, PROFIT_SCORING_WEIGHTS_P3

print("Veri yükleniyor...")
try:
    df = load_data(DB_PATH)
    print(f"Toplam {len(df)} veri yüklendi.")
    
    # --- 3-WAY SPLIT STRATEJİSİ ---
    # 1. Base Train (%70): Temel modelleri eğitmek için.
    # 2. Meta Train (%15): Temel modellerin tahminlerini alıp Meta-Learner'ı eğitmek için.
    # 3. Test (%15): Final simülasyon ve değerlendirme için (Hiçbir model burayı görmez).
    
    total_len = len(df)
    base_train_end = int(total_len * 0.70)
    meta_train_end = int(total_len * 0.85)
    
    print(f"--- Veri Ayrımı ---")
    print(f"Base Train: 0 - {base_train_end} ({base_train_end} örnek)")
    print(f"Meta Train: {base_train_end} - {meta_train_end} ({meta_train_end - base_train_end} örnek)")
    print(f"Test (Sim): {meta_train_end} - {total_len} ({total_len - meta_train_end} örnek)")
    
except Exception as e:
    print(f"Hata: {e}")
    print("Lütfen jetx.db dosyasının proje klasöründe olduğundan emin olun.")

In [ ]:
# 2.5. Anomaly Detector (The Shield) Eğitimi
from jetx_project.model_anomaly import train_anomaly_detector, save_anomaly_detector

if 'df' in locals() and len(df) > 500:
    print("Anomaly Detector (The Shield) eğitiliyor...")
    # Base Train verisiyle eğit
    train_values_anomaly = df['value'].values[:base_train_end]
    anomaly_model = train_anomaly_detector(train_values_anomaly)
    save_anomaly_detector(anomaly_model, output_dir='models')
    print("Anomaly Detector tamamlandı.")
else:
    print("Yeterli veri yok.")


In [ ]:
# 3. Psikolojik Modeller (HMM) Eğitimi
print("Psikolojik Modeller (HMM) eğitiliyor...")

# Fallback: If hmmlearn is missing, install it here.
try:
    from hmmlearn import hmm
except ImportError:
    print("⚠️ hmmlearn bulunamadı, şimdi yükleniyor...")
    import sys
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "hmmlearn==0.3.2"])
    print("hmmlearn yüklendi.")

from jetx_project.model_hmm import train_categorical_hmm, predict_categorical_hmm_states_causal, save_hmm_model

if 'df' in locals() and len(df) > 0:
    # Sadece Base Train verisiyle eğit
    train_values = df['value'].values[:base_train_end]
    
    # HMM Eğitimi (Categorical)
    hmm_model, hmm_map, hmm_bins = train_categorical_hmm(train_values)
    save_hmm_model(hmm_model, hmm_map, bins=hmm_bins, output_dir='models')
    
    # Tüm veri seti için durum tahmini (Feature olarak kullanılacak)
    # CAUSAL PREDICTION: Geleceği görmeden, kayan pencere ile tahmin et.
    all_values = df['value'].values
    hmm_states_mapped = predict_categorical_hmm_states_causal(hmm_model, all_values, hmm_map, bins=hmm_bins, window_size=200)
    
    print("HMM Durumları (Causal) tüm veri için belirlendi.")
    print(f"Durum Dağılımı: {np.bincount(hmm_states_mapped)}")
else:
    print("Veri yok, HMM eğitilemedi.")

In [ ]:
# 4. OPTUNA İLE HİPERPARAMETRE OPTİMİZASYONU (Base Train Üzerinde)
from jetx_project.optimization import optimize_catboost, optimize_lightgbm, optimize_mlp, optimize_lstm
from jetx_project.model_a import prepare_model_a_data

if 'df' in locals() and len(df) > 500:
    print("Optimizasyon için veri hazırlanıyor...")
    # Data Prep
    X, y_p15, y_p3, y_x = prepare_model_a_data(df['value'].values, hmm_states_mapped)
    
    # Filter Base Train
    limit_idx = base_train_end
    mask_opt = (X.index < limit_idx)
    X_train_opt = X[mask_opt]
    y_p15_train_opt = y_p15[mask_opt]
    y_p3_train_opt = y_p3[mask_opt]
    
    print(f"Optimizasyon Veri Seti: {len(X_train_opt)} satır")
    
    # --- CatBoost ---
    print("\n=== CatBoost Optimizasyonu ===")
    print("-> P1.5:")
    best_params_catboost_p15 = optimize_catboost(X_train_opt, y_p15_train_opt, n_trials=15, scoring_params=PROFIT_SCORING_WEIGHTS)
    print("-> P3.0:")
    best_params_catboost_p3 = optimize_catboost(X_train_opt, y_p3_train_opt, n_trials=15, scoring_params=PROFIT_SCORING_WEIGHTS_P3)
    
    # --- LightGBM ---
    print("\n=== LightGBM Optimizasyonu ===")
    print("-> P1.5:")
    best_params_lgbm_p15 = optimize_lightgbm(X_train_opt, y_p15_train_opt, n_trials=15, scoring_params=PROFIT_SCORING_WEIGHTS)
    print("-> P3.0:")
    best_params_lgbm_p3 = optimize_lightgbm(X_train_opt, y_p3_train_opt, n_trials=15, scoring_params=PROFIT_SCORING_WEIGHTS_P3)
    
    # --- MLP ---
    print("\n=== MLP Optimizasyonu ===")
    print("-> P1.5:")
    best_params_mlp_p15 = optimize_mlp(X_train_opt, y_p15_train_opt, n_trials=30, scoring_params=PROFIT_SCORING_WEIGHTS)
    print("-> P3.0:")
    best_params_mlp_p3 = optimize_mlp(X_train_opt, y_p3_train_opt, n_trials=30, scoring_params=PROFIT_SCORING_WEIGHTS_P3)
    
    # --- LSTM (Requires Sequence Prep) ---
    # --- LSTM (Requires Sequence Prep) ---
    
    print("\n✅ (CatBoost/LGBM/MLP) Optimizasyonu Tamamlandı.")
else:
    print("Yeterli veri yok.")


In [ ]:
# 4.2 LSTM OPTİMİZASYONU
from jetx_project.optimization import optimize_lstm
import numpy as np
if 'df' in locals() and len(df) > 500:
    print("\n=== LSTM Optimizasyonu ===")
    
    # optimization.py handles scaler isolation internally to prevent Data Leakage.
    # We just pass the RAW chronological values.
    train_values_base = df['value'].values[:base_train_end]
    
    print("-> P1.5:")
    best_params_lstm_p15 = optimize_lstm(train_values_base, n_trials=10, scoring_params=PROFIT_SCORING_WEIGHTS)
    
    print("-> P3.0:")
    best_params_lstm_p3 = optimize_lstm(train_values_base, n_trials=10, scoring_params=PROFIT_SCORING_WEIGHTS_P3)

    print("\n✅ LSTM Optimizasyonu Tamamlandı. En iyi parametreler kayıt edildi.")
else:
    print("Yeterli veri yok.")


In [ ]:
# 5. Model A (CatBoost) Eğitimi (Base Train)
from jetx_project.model_a import train_model_a, save_models
if 'best_params_catboost_p15' not in locals(): best_params_catboost_p15 = None
if 'best_params_catboost_p3' not in locals(): best_params_catboost_p3 = None
from catboost import CatBoostClassifier

if 'df' in locals() and len(df) > 500:
    print("Model A (CatBoost) eğitiliyor...")
    
    # Sadece Base Train verisini kullan
    limit_idx = base_train_end
    mask_train = (X.index < limit_idx)
    
    X_train_base = X[mask_train]
    y_p15_base = y_p15[mask_train]
    y_p3_base = y_p3[mask_train]
    y_x_base = y_x[mask_train]
    
    # train_model_a kendi içinde %15 validasyon ayırır (Base Train'in %15'i)
    modelA_p15, modelA_p3, modelA_x = train_model_a(X_train_base, y_p15_base, y_p3_base, y_x_base, params_p15=best_params_catboost_p15, params_p3=best_params_catboost_p3, scoring_params_p15=PROFIT_SCORING_WEIGHTS, scoring_params_p3=PROFIT_SCORING_WEIGHTS_P3)
    save_models(modelA_p15, modelA_p3, modelA_x, output_dir='models')
    print("Model A tamamlandı.")
else:
    print("Yeterli veri yok.")


In [ ]:
# 6. Model B (k-NN) Eğitimi (Base Train)
from jetx_project.model_b import build_memory, train_model_b, save_memory

if 'df' in locals() and len(df) > 500:
    print("Model B (Hafıza) oluşturuluyor...")
    
    # Sadece Base Train verisiyle hafıza oluştur
    train_values_b = df['value'].values[:base_train_end]
    patterns, targets = build_memory(train_values_b)
    
    nbrs, pca = train_model_b(patterns)
    
    save_memory(nbrs, pca, patterns, targets, output_dir='models')
    print("Model B tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
if 'best_params_lstm_p15' not in locals(): best_params_lstm_p15 = None
if 'best_params_lstm_p3' not in locals(): best_params_lstm_p3 = None
# 7. Model C (LSTM) Eğitimi (Base Train)
from jetx_project.model_lstm import train_model_lstm, save_lstm_models

if 'df' in locals() and len(df) > 500:
    print("Model C (LSTM) eğitiliyor...")
    
    # Sadece Base Train verisi
    train_values_c = df['value'].values[:base_train_end]
    
    modelC_p15, modelC_p3, scaler_lstm = train_model_lstm(train_values_c, params_p15=best_params_lstm_p15, params_p3=best_params_lstm_p3, scoring_params_p15=PROFIT_SCORING_WEIGHTS, scoring_params_p3=PROFIT_SCORING_WEIGHTS_P3)
    save_lstm_models(modelC_p15, modelC_p3, scaler_lstm, output_dir='models')
    print("Model C tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
if 'best_params_lgbm_p15' not in locals(): best_params_lgbm_p15 = None
if 'best_params_lgbm_p3' not in locals(): best_params_lgbm_p3 = None
# 8. Model D (LightGBM) Eğitimi (Base Train)
from jetx_project.model_lightgbm import train_model_lightgbm, save_lightgbm_models

if 'df' in locals() and len(df) > 500:
    print("Model D (LightGBM) eğitiliyor...")
    modelD_p15, modelD_p3 = train_model_lightgbm(X_train_base, y_p15_base, y_p3_base, params_p15=best_params_lgbm_p15, params_p3=best_params_lgbm_p3, scoring_params_p15=PROFIT_SCORING_WEIGHTS, scoring_params_p3=PROFIT_SCORING_WEIGHTS_P3)
    save_lightgbm_models(modelD_p15, modelD_p3, output_dir='models')
    print("Model D tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
if 'best_params_mlp_p15' not in locals(): best_params_mlp_p15 = None
if 'best_params_mlp_p3' not in locals(): best_params_mlp_p3 = None
# 9. Model E (MLP) Eğitimi (Base Train)
from jetx_project.model_mlp import train_model_mlp, save_mlp_models

if 'df' in locals() and len(df) > 500:
    print("Model E (MLP) eğitiliyor...")
    modelE_p15, modelE_p3, mlp_cols = train_model_mlp(X_train_base, y_p15_base, y_p3_base, params_p15=best_params_mlp_p15, params_p3=best_params_mlp_p3, scoring_params_p15=PROFIT_SCORING_WEIGHTS, scoring_params_p3=PROFIT_SCORING_WEIGHTS_P3)
    save_mlp_models(modelE_p15, modelE_p3, mlp_cols, output_dir='models')
    print("Model E tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
import os, sys
project_dir = '/content/newproje'
if os.path.exists(project_dir) and project_dir not in sys.path:
    sys.path.append(project_dir)
if os.path.exists(project_dir):
    %cd /content/newproje
# 9.1. Model F (Transformer - The Attention) Eğitimi (Base Train)
from jetx_project.model_transformer import train_model_transformer, save_transformer_models
if 'df' in locals() and len(df) > 500:
    print("Model F (Transformer) eğitiliyor...")
    # Base Train verisi
    train_values_f = df['value'].values[:base_train_end]
    model_transformer, scaler_transformer = train_model_transformer(train_values_f)
    save_transformer_models(model_transformer, scaler_transformer, output_dir='models')
    print("Model F (Transformer) tamamlandı.")
else:
    print("Yeterli veri yok.")


In [ ]:
# 9.5. ENSEMBLE STACKING (Meta-Learner) Eğitimi (Meta Train)
from jetx_project.ensemble import prepare_meta_features, train_meta_learner, save_meta_learner, predict_meta
from jetx_project.model_lstm import create_sequences
# Reloading Imports (Safety Net)
from jetx_project.model_a import load_models as load_model_a
from jetx_project.model_b import load_memory as load_model_b
from jetx_project.model_lstm import load_lstm_models
from jetx_project.model_lightgbm import load_lightgbm_models
from jetx_project.model_mlp import load_mlp_models
from jetx_project.model_transformer import load_transformer_models
from jetx_project.model_hmm import load_hmm_model, predict_categorical_hmm_states

if 'df' in locals() and len(df) > 2000:
    print("--- Ensemble Stacking (Meta-Learner) Eğitiliyor ---")
    
    # --- SAFETY CHECK: RELOAD MODELS IF MISSING ---
    # Colab runtime might have cleared variables. Reload from disk if needed.
    if 'modelA_p15' not in locals():
        print("⚠️ Variables missing. Reloading models from disk...")
        modelA_p15, modelA_p3, modelA_x = load_model_a('models')
        
        nbrs, pca, targets, _ = load_model_b('models') # patterns not needed for prediction
        
        modelC_p15, modelC_p3, scaler_lstm = load_lstm_models('models')
        
        modelD_p15, modelD_p3 = load_lightgbm_models('models')
        
        modelE_p15, modelE_p3, mlp_cols = load_mlp_models('models')
        
        model_transformer, scaler_transformer = load_transformer_models('models')
        
        hmm_model, hmm_map, hmm_bins = load_hmm_model('models')
        # Re-calc hmm_states_mapped if not present
        if 'hmm_states_mapped' not in locals():
             print("Recalculating HMM states...")
             # Use categorical prediction since we loaded categorical model
             # Note: The training cell used causal prediction, but for simple reload we can use standard
             # or simple re-run causal if 'predict_categorical_hmm_states_causal' imported.
             # For simplicity here, let's use the standard predict function or assume it's loaded.
             # Better: just use what we have.
             from jetx_project.model_hmm import predict_categorical_hmm_states_causal
             all_values = df['value'].values
             hmm_states_mapped = predict_categorical_hmm_states_causal(hmm_model, all_values, hmm_map, bins=hmm_bins, window_size=200)
        
        print("✅ Models reloaded successfully.")
    # -----------------------------------------------

    meta_start_idx = base_train_end
    meta_end_idx = meta_train_end
    print(f"Meta Train Aralığı: {meta_start_idx} - {meta_end_idx}")
    # 1. Meta Train Seti İçin Tahminleri Topla
    # Model A (CatBoost)
    from jetx_project.model_a import prepare_model_a_data
    X_meta, _, _, _ = prepare_model_a_data(df['value'].values, hmm_states_mapped, start_index=meta_start_idx)
    X_meta = X_meta[X_meta.index < meta_end_idx]
    # Hedefler
    y_meta_true_15 = (df['value'].values[meta_start_idx+1 : meta_end_idx+1] >= 1.5).astype(int)
    # Uzunluk Kontrolü
    min_len = min(len(X_meta), len(y_meta_true_15))
    X_meta = X_meta.iloc[:min_len]
    y_meta_true_15 = y_meta_true_15[:min_len]
    print(f"Meta Train Örnek Sayısı: {min_len}")
    preds_a_meta = modelA_p15.predict_proba(X_meta)[:, 1]
    # Model B (k-NN)
    preds_b_meta = []
    from jetx_project.model_b import create_pattern_vector, predict_model_b
    pca_obj = locals().get('pca', None) 
    for idx in X_meta.index:
        pat = create_pattern_vector(df['value'].values, idx)
        if pat is not None:
            p15, _, _ = predict_model_b(nbrs, pca_obj, targets, pat)
            preds_b_meta.append(p15)
        else:
            preds_b_meta.append(0.5)
    preds_b_meta = np.array(preds_b_meta)
    # Model C (LSTM)
    seq_len = 200
    lstm_start_offset = X_meta.index[0] - seq_len
    lstm_end_offset = X_meta.index[-1]
    meta_values_extended = df['value'].values[lstm_start_offset : lstm_end_offset + 1]
    meta_values_scaled = scaler_lstm.transform(meta_values_extended.reshape(-1, 1))
    X_lstm_meta, _, _, _ = create_sequences(meta_values_scaled, meta_values_extended, seq_len)
    preds_c_meta = modelC_p15.predict(X_lstm_meta).flatten()
    # Model D (LightGBM)
    preds_d_meta = modelD_p15.predict_proba(X_meta)[:, 1]
    # Model E (MLP)
    X_meta_mlp = X_meta[mlp_cols]
    preds_e_meta = modelE_p15.predict_proba(X_meta_mlp)[:, 1]
    # Model F (Transformer)
    meta_values_scaled_trans = scaler_transformer.transform(meta_values_extended.reshape(-1, 1))
    X_trans_meta, _, _, _ = create_sequences(meta_values_scaled_trans, meta_values_extended, seq_len)
    preds_f_meta = model_transformer.predict(X_trans_meta)
    preds_f_meta_p15 = preds_f_meta[0].flatten()
    # HMM States
    hmm_meta = hmm_states_mapped[X_meta.index]
    # 2. Meta-Features Hazırla
    final_len = min(len(preds_a_meta), len(preds_b_meta), len(preds_c_meta), len(preds_d_meta), len(preds_e_meta), len(preds_f_meta_p15))
    meta_values_raw = df['value'].values[meta_start_idx : meta_end_idx]
    meta_values_raw = meta_values_raw[:final_len]
    meta_X_train = prepare_meta_features(
        preds_a_meta[:final_len], 
        preds_b_meta[:final_len], 
        preds_c_meta[:final_len], 
        preds_d_meta[:final_len], 
        preds_e_meta[:final_len], 
        hmm_meta[:final_len],
        values=meta_values_raw,
        preds_transformer=preds_f_meta_p15[:final_len]
    )
    y_meta_train = y_meta_true_15[:final_len]
    # 3. Eğit
    meta_model, meta_scaler = train_meta_learner(meta_X_train, y_meta_train)
    save_meta_learner(meta_model, meta_scaler, output_dir='models')
    print("Meta-Learner eğitildi ve kaydedildi.")
else:
    print("Yeterli veri yok.")


In [ ]:
# 9.8. RL Agent (The Strategist) Eğitimi
from jetx_project.model_rl import train_rl_agent, save_rl_agent

if 'meta_model' in locals():
    print("RL Agent (The Strategist) eğitiliyor...")
    
    # Meta-Learner tahminlerini alalım
    meta_probs = predict_meta(meta_model, meta_scaler, meta_X_train)
    
    # DataFrame oluştur
    rl_train_df = pd.DataFrame({
        'value': meta_values_raw, # Gerçek değerler
        'prob_1.5': meta_probs,
        'hmm_state': hmm_meta[:final_len] # HMM durumları
    })
    
    rl_agent = train_rl_agent(rl_train_df)
    save_rl_agent(rl_agent, output_dir='models')
    print("RL Agent tamamlandı.")
else:
    print("Meta-Learner hazır değil.")


In [ ]:
# 10. BÜYÜK FİNAL: Simülasyon (Test Seti Üzerinde)
# Strateji Güncellemesi: Sadece Eşik 0.70 (HMM Filtresi Yok)

print("--- BÜYÜK FİNAL: Simülasyon Başlıyor (Test Seti) ---")

test_start_idx = meta_train_end
test_end_idx = len(df) - 1
test_len = test_end_idx - test_start_idx
print(f"Test Seti Başlangıç: {test_start_idx}, Uzunluk: {test_len}")

if test_len > 200:
    # Test verisini hazırla
    sim_df = df.iloc[test_start_idx:test_end_idx].copy()
    sim_df = sim_df.reset_index(drop=True)
    
    print("Modeller tahmin ediyor...")
    
    # 1. Toplu Tahmin (Batch Prediction)
    
    # Model A (CatBoost)
    from jetx_project.model_a import prepare_model_a_data
    X_test_a, _, _, _ = prepare_model_a_data(df['value'].values, hmm_states_mapped, start_index=test_start_idx)
    min_len = min(len(X_test_a), len(sim_df))
    X_test_a = X_test_a.iloc[:min_len]
    sim_df = sim_df.iloc[:min_len]
    
    sim_df['prob_A_1.5'] = modelA_p15.predict_proba(X_test_a)[:, 1]
    
    # Model B (k-NN) - VECTORIZED / BATCH
    print("Model B (k-NN) Batch Prediction...")
    from jetx_project.model_b import create_pattern_vector, predict_model_b
    pca_obj = locals().get('pca', None)
    
    patterns_list = []
    valid_indices = []
    for idx in X_test_a.index:
        pat = create_pattern_vector(df['value'].values, idx)
        if pat is not None:
            patterns_list.append(pat)
            valid_indices.append(len(patterns_list)-1)
        else:
            patterns_list.append(np.zeros(300))
            
    import numpy as np
    X_patterns = np.array(patterns_list)
    preds_b_p15, _, _ = predict_model_b(nbrs, pca_obj, targets, X_patterns)
    sim_df['prob_B_1.5'] = preds_b_p15
    
    # Model C (LSTM)
    seq_len = 200
    lstm_start = X_test_a.index[0] - seq_len
    test_values_extended = df['value'].values[lstm_start : X_test_a.index[-1] + 1]
    test_values_scaled = scaler_lstm.transform(test_values_extended.reshape(-1, 1))
    X_lstm, _, _, _ = create_sequences(test_values_scaled, test_values_extended, seq_len)
    sim_df['prob_C_1.5'] = modelC_p15.predict(X_lstm, verbose=0).flatten()[:min_len]
    
    # Model D (LightGBM)
    sim_df['prob_D_1.5'] = modelD_p15.predict_proba(X_test_a)[:, 1]
    
    # Model E (MLP)
    X_test_mlp = X_test_a[mlp_cols]
    sim_df['prob_E_1.5'] = modelE_p15.predict_proba(X_test_mlp)[:, 1]

    # Model F (Transformer)
    meta_values_scaled_trans = scaler_transformer.transform(test_values_extended.reshape(-1, 1))
    X_trans_test, _, _, _ = create_sequences(meta_values_scaled_trans, test_values_extended, seq_len)
    preds_f_test = model_transformer.predict(X_trans_test, verbose=0)
    sim_df['prob_F_1.5'] = preds_f_test[0].flatten()[:min_len]
    
    # HMM Feature for Filter (Gözlem için tutuyoruz ama şart değil)
    hmm_test = hmm_states_mapped[X_test_a.index]
    sim_df['hmm_state'] = hmm_test
    
    # 2. Meta-Learner Tahmini
    from jetx_project.ensemble import prepare_meta_features, predict_meta
    
    meta_X_test = prepare_meta_features(
        sim_df['prob_A_1.5'].values,
        sim_df['prob_B_1.5'].values,
        sim_df['prob_C_1.5'].values,
        sim_df['prob_D_1.5'].values,
        sim_df['prob_E_1.5'].values,
        hmm_test,
        values=sim_df['value'].values,
        preds_transformer=sim_df['prob_F_1.5'].values
    )
    
    sim_df['meta_prob'] = meta_model.predict_proba(meta_X_test)[:, 1]
    
    # 3. Simülasyon
    initial_balance = 1000
    balance = initial_balance
    balance_history = [balance]
    bet_amount = 10
    
    wins = 0
    losses = 0
    
    print("\n--- OPTİMİZE EDİLMİŞ Simülasyon Sonuçları ---")
    print("Kurallar: Güven > 0.70 (HMM Filtresi YOK)")
    
    for i, row in sim_df.iterrows():
        prob = row['meta_prob']
        real_outcome = row['value']
        
        # Strateji: Sadece Prob > 0.70
        if prob > 0.70:
            if real_outcome >= 1.50:
                profit = bet_amount * 0.50
                balance += profit
                wins += 1
            else:
                balance -= bet_amount
                losses += 1
        
        balance_history.append(balance)
        
    print(f"Başlangıç Kasa: {initial_balance}")
    print(f"Bitiş Kasa: {balance:.2f}")
    print(f"Kar/Zarar: {balance - initial_balance:.2f}")
    print(f"Toplam Bahis: {wins + losses}")
    print(f"Kazanma Oranı: {wins / (wins + losses) if (wins+losses) > 0 else 0:.2%}")
    
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 6))
    plt.plot(balance_history)
    plt.title('Simülasyon Kasa Grafiği (Eşik 0.70)')
    plt.xlabel('Oyun Sayısı')
    plt.ylabel('Bakiye')
    plt.axhline(y=initial_balance, color='r', linestyle='--')
    plt.show()
    
else:
    print("Test seti çok kısa.")


In [ ]:
# 11. Modelleri Yedekle (Google Drive & Local)
import os
import datetime
import shutil
from google.colab import drive, files

print("--- Modeller Yedekleniyor ---")

# 1. Zip Oluştur
print("Modeller sıkıştırılıyor (models.zip)...")
!zip -r models.zip models/

# 2. Google Drive Yedekleme
try:
    print("Google Drive bağlanıyor...")
    drive.mount('/content/drive')
    
    # Zaman damgalı klasör adı
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M")
    backup_folder_name = f"JetX_Backup_{timestamp}"
    backup_path = f"/content/drive/MyDrive/{backup_folder_name}"
    
    if not os.path.exists(backup_path):
        os.makedirs(backup_path)
        print(f"Yedekleme klasörü oluşturuldu: {backup_path}")
        
    # Kopyala
    shutil.copy("models.zip", f"{backup_path}/models.zip")
    print(f"✅ BAŞARILI: Modeller Drive'a yedeklendi -> {backup_path}/models.zip")
    
except Exception as e:
    print(f"⚠️ UYARI: Google Drive yedeklemesi başarısız oldu: {e}")
    print("Yerel indirme deneniyor...")

# 3. Yerel İndirme (Her durumda sunulur)
try:
    files.download('models.zip')
    print("✅ İndirme başlatıldı.")
except Exception as e:
    print(f"İndirme hatası: {e}")
